In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np

In [3]:
# Define the EDSR model
class EDSR(nn.Module):
    def __init__(self, scale_factor=2, num_blocks=4, num_channels=128):
        super(EDSR, self).__init__()
        # Initial convolution layer
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size=3, padding=1)
        # Residual blocks
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_channels) for _ in range(num_blocks)]
        )
        # Another convolution layer
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        # Upsampling layer
        self.upsample = nn.Sequential(
            nn.Conv2d(num_channels, num_channels * scale_factor ** 2, kernel_size=3, padding=1),
            nn.PixelShuffle(scale_factor)
        )
        # Final convolution layer
        self.conv3 = nn.Conv2d(num_channels, 3, kernel_size=3, padding=1)

    def forward(self, x):
        out = self.conv1(x)
        residual = out
        out = self.res_blocks(out)
        out = self.conv2(out)
        out += residual  # Skip connection
        out = self.upsample(out)
        out = self.conv3(out)
        return out

class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        out += residual  # Skip connection
        return out

In [3]:
class LRHRPairDataset(Dataset):
    def __init__(self, root_dir, lr_transform=None, hr_transform=None, scale_factor=2):
        self.scale_factor = scale_factor
        self.root_dir = root_dir
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform
        self.image_filenames = [f for f in os.listdir(root_dir)]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        image_path = os.path.join(self.root_dir + "\\", image_filename)
        # Load the HR image
        hr_image = Image.open(image_path).convert("RGB")
        lr_image = hr_image.resize((hr_image.width // self.scale_factor, hr_image.height // self.scale_factor), Image.BICUBIC)
        if self.lr_transform:
            lr_image = self.lr_transform(lr_image)
        if self.hr_transform:
            hr_image = self.hr_transform(hr_image)
        return lr_image, hr_image


In [4]:
# Define data transforms (you can customize these)
lr_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
])
hr_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
])

In [5]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 20
batch_size = 4

In [6]:
dataset_path = r"D:\COMPUTER ENGINEERING\SEMESTER 7\ML\Project\train_LR"
# Create a DataLoader for LR-HR pairs
dataset = LRHRPairDataset(root_dir=dataset_path, lr_transform=lr_transform, hr_transform=hr_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EDSR().to(device)
model.train()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [8]:
for epoch in range(num_epochs):
    print(f"Training Epoch {epoch + 1}..", end="\n")
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        # Load and preprocess data
        input_images, target_images = batch
        input_images = input_images.cuda()
        target_images = target_images.cuda()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(input_images)
        # Compute the loss
        loss = criterion(outputs, target_images)
        del input_images
        del target_images
        del outputs
        # Backpropagation
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {total_loss / len(dataloader)}", end="\n")
torch.save(model.state_dict(), 'EDSRv4.50.pth')

Training Epoch 1..
Epoch [1/20] Loss: 0.07188567243167199
Training Epoch 2..
Epoch [2/20] Loss: 0.00566643635625951
Training Epoch 3..
Epoch [3/20] Loss: 0.00551321601960808
Training Epoch 4..
Epoch [4/20] Loss: 0.004954555051517673
Training Epoch 5..
Epoch [5/20] Loss: 0.0048339261912042275
Training Epoch 6..
Epoch [6/20] Loss: 0.004764156247256323
Training Epoch 7..
Epoch [7/20] Loss: 0.004728495594463311
Training Epoch 8..
Epoch [8/20] Loss: 0.004983787848614156
Training Epoch 9..
Epoch [9/20] Loss: 0.00470901706663426
Training Epoch 10..
Epoch [10/20] Loss: 0.004656810942688026
Training Epoch 11..
Epoch [11/20] Loss: 0.004645269777392968
Training Epoch 12..
Epoch [12/20] Loss: 0.004633926360402257
Training Epoch 13..
Epoch [13/20] Loss: 0.004627035671146587
Training Epoch 14..
Epoch [14/20] Loss: 0.004660168158588931
Training Epoch 15..
Epoch [15/20] Loss: 0.004669427409535274
Training Epoch 16..
Epoch [16/20] Loss: 0.004770465261535719
Training Epoch 17..
Epoch [17/20] Loss: 0.004

## EXPORTING MODEL

In [5]:
model_scripted = torch.jit.script(model)
model_scripted.save('./saved_models/EDSRx2v4.20.pt')